#Transformaciones sobre el dataframe de Series

#1.Definir listas para guardar errores

In [0]:
log_error_list = []

#2.Instalación de paquetes e importación de librerías

In [0]:
#Se importan las librerías a utilizar 
import requests
import pprint
import pandas as pd
import json
import glob
import sys
from datetime import date, time, datetime
import numpy as np

#3.Lectura de la sabana de datos inicial

In [0]:
if len(log_error_list)>0:
    print("Previous Error")
else:
    try:
      ##Se lee la sabana inicial de datos 
      Sabana_inicial = pd.read_csv("/dbfs/mnt/processed/Sabana_inicial/Sabana_inicial.csv", sep=',')
      
    except Exception as e:
        print("Failed! Due to: {}".format(str(e)))
        log_error_list.append("Error: {}".format(str(e)))

#4.Dataframe de embedded_show por el _embedded.show.network.id

In [0]:
if len(log_error_list)>0:
    print("Previous Error")
else:
    try:
        
        Series=Sabana_inicial[['id', 'url', 'name', 'season', 'number', 'type','airdate', 'airtime','airstamp', 'runtime', 'image', 'summary',
                               'rating.average','_links.self.href','_embedded.show.dvdCountry','_embedded.show.externals.tvrage', '_embedded.show.externals.thetvdb',
                              '_embedded.show.externals.imdb', '_embedded.show.image.medium',
                              '_embedded.show.image.original', '_embedded.show.summary',
                              '_embedded.show.updated', '_embedded.show._links.self.href',
                              '_embedded.show._links.previousepisode.href', 'image.medium',
                              'image.original','_embedded.show._links.nextepisode.href',
                               '_embedded.show.webChannel','_embedded.show.image', '_embedded.show.webChannel.country',
                              '_embedded.show.dvdCountry.name', '_embedded.show.dvdCountry.code',
                               '_embedded.show.dvdCountry.timezone',
                               '_embedded.show.id','_embedded.show.webChannel.id',
                               '_embedded.show.network.id','_embedded.show.averageRuntime',
                               '_embedded.show.genres','_embedded.show.schedule.days']]
        
        #Definición de tipo de datos 
        Series = Series.astype({'id':'str', '_embedded.show.id':'str', 
                              '_embedded.show.webChannel.id':'str',
                              '_embedded.show.network.id':'str',
                              'airdate':'datetime64',
                              'airtime':'datetime64',
                              'airstamp':'datetime64',
                              '_embedded.show.updated':'str' })
        

        #Filtrar los registros que no tengan id
        Series = Series[Series['id'].notnull()]
        
        
        #Eliminar duplicado por el id
        #Series = Series.drop_duplicates(subset=['id'])
        
        #Renombrar columnas  (Si hay tiempo buscar una forma más inteligente para hacer el cambio de nombre )
        Series = Series.rename(columns={'rating.average':'rating_average',
                                          '_links.self.href':'links_self_href',
                                          'embedded.show.dvdCountry':'_embedded_show_dvdCountry',
                                          '_embedded.show.externals.tvrage':'embedded_show_externals_tvrage',
                                          '_embedded.show.externals.thetvdb':'embedded_show_externals_thetvdb',
                                          '_embedded.show.externals.imdb':'embedded_show_externals_imdb',
                                          '_embedded.show.image.medium':'embedded_show_image.medium',
                                          '_embedded.show.image.original':'embedded_show_image_original',
                                          '_embedded.show.summary':'embedded_show_summary',
                                          '_embedded.show.updated':'embedded_show_updated',
                                          '_embedded.show._links.self.href':'embedded_show_links_self_href',
                                          '_embedded.show._links.previousepisode.href':'embedded_show_links_previousepisode_href',
                                          'image.medium':'image_medium',
                                          'image.original':'image_original',
                                          '_embedded.show._links.nextepisode.href':'embedded_show_links_nextepisode_href',
                                          '_embedded.show.webChannel':'embedded_show_webChannel', 
                                          '_embedded.show.image':'embedded_show_image', 
                                          '_embedded.show.webChannel.country':'embedded_show_webChannel_country',
                                          '_embedded.show.dvdCountry.name':'embedded_show_dvdCountry_name',
                                          '_embedded.show.dvdCountry.code':'embedded_show_dvdCountry_code',
                                          '_embedded.show.dvdCountry.timezone':'embedded_show_dvdCountry_timezone',
                                          '_embedded.show.id':'embedded_show_id',
                                          '_embedded.show.webChannel.id':'embedded_show_webChannel_id',
                                          '_embedded.show.network.id':'embedded_show_network_id',
                                          '_embedded.show.averageRuntime':'embedded_show_averageRuntime',
                                          '_embedded.show.genres':'embedded_show_genres', 
                                          '_embedded.show.schedule.days':'embedded_show_schedule_days',
                                          '_embedded.show.dvdCountry':'embedded_show_dvdCountry',
                                          '_embedded.show.image.medium':'embedded_show_image_medium'})
        #Reiniciar el índice
        Series.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
      
    except Exception as e:
        print("Failed! Due to: {}".format(str(e)))
        log_error_list.append("Error: {}".format(str(e)))

#5.Construcción de dimensión de Género y adecuación de la tabla de Series

##5.1 Limpieza de la tabla y expasión de la columna Género

In [0]:
if len(log_error_list)>0:
    print("Previous Error")
else:
    try:
        ##Quitar corchetes de las listas de la columna embedded_show_genres (Si hay tiempo buscar una forma más inteligente de hacer este paso)
        Series.embedded_show_genres = Series.embedded_show_genres.str.replace("[",'')
        Series.embedded_show_genres = Series.embedded_show_genres.str.replace("]",'')
        Series.embedded_show_genres = Series.embedded_show_genres.str.replace("'",'')

        ##Quitar corchetes de las listas de la columna embedded_show_schedule_days (Si hay tiempo buscar una forma más inteligente de hacer este paso)
        Series.embedded_show_schedule_days = Series.embedded_show_schedule_days.str.replace("[",'')
        Series.embedded_show_schedule_days = Series.embedded_show_schedule_days.str.replace("]",'')
        Series.embedded_show_schedule_days = Series.embedded_show_schedule_days.str.replace("'",'')

        #Aplicar función de limpieza para convertir a mayúsculas y miniscúlas 
        def limpieza(columna):
          x=columna.strip()
          y=x.upper()
          return y

        Series['embedded_show_genres']=Series['embedded_show_genres'].apply(limpieza)
        Series['embedded_show_schedule_days']=Series['embedded_show_schedule_days'].apply(limpieza)

        #Se calcula el numero de generos 
        num_generos = Series.embedded_show_genres.str.split(',', expand=True).shape[1]

        #Se crea un ciclo que rellene una lista con el numero de generos que tiene el dataframe 
        list=[None]*num_generos

        for i in range(num_generos):
            a=i
            list[i]="Genero_"+str(a)

        #Hacer un split por la coma en la columna embedded_show_genres
        Series[list]=Series.embedded_show_genres.str.split(',',expand=True)


        """
        Por un tema de tiempo solo haré la homologación para el género y la construcción de esta dimensión para poder contar, el número de series por género y tener un dato bien calculado, ya que una serie puede tener varios géneros. Para el caso de la columna embedded_show_schedule_days, se podría hacer el mismo ejercicio, que se hará a continuación, ya que una serie, pude estar transimitiendose en varios dias. Entonces, nos podría interesar un indicador, de que serie o series se ven mas o menos segun el dia de la semana. 

        #Se repite el proceso anterior para embedded_show_schedule_days

        #Se calcula el numero de dias 
        num_dias= Series.embedded_show_schedule_days.str.split(',', expand=True).shape[1]

        #Se crea un ciclo que rellene una lista con el numero de dias que tiene el dataframe 
        list1=[None]*num_dias

        for i in range(num_dias):
            a=i
            list1[i]="Day "+str(a)

        #Hacer un split por la coma en la columna embedded_show_schedule_days
        Series[list1]=Series.embedded_show_schedule_days.str.split(',',expand=True)
        """

        #Eliminar columnas 
        Series =Series.drop(['embedded_show_genres'], axis=1)
      
    except Exception as e:
        print("Failed! Due to: {}".format(str(e)))
        log_error_list.append("Error: {}".format(str(e)))

<command-2332635609408710>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
 Series.embedded_show_genres = Series.embedded_show_genres.str.replace("[",'')
<command-2332635609408710>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
 Series.embedded_show_genres = Series.embedded_show_genres.str.replace("]",'')
<command-2332635609408710>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
 Series.embedded_show_schedule_days = Series.embedded_show_schedule_days.str.replace("[",'')
<command-2332635609408710>:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
 Series.embedded_show_schedule_days = Series.embedded_show_schedule_days.str.replace("]",'')

##5.2 Construcción dimensión de género

In [0]:
if len(log_error_list)>0:
    print("Previous Error")
else:
    try:
        #Nos quedamos con las columnas de género
        Generos=Series[list]

        #Declaramos una lista con el tamaño del número de géneros  y que siempre el nombre sea genero

        new_cols = ['Genero']*num_generos 

        #Cambiamos el nombre de las columnas del dataframe para que todas sea Genero
        new_names_map = {Generos.columns[i]:new_cols[i] for i in range(len(new_cols))} 

        Generos.rename(new_names_map, axis=1, inplace=True)

        #generamos una lista para guardar las columnas de los generos
        list_Genero=[None]*num_generos

        #iteramos en el dataframes para guardar una columna en cada posición de la lista
        for i in range(num_generos):
            list_Genero[i]= Generos.iloc[:,i]

        #Hacemos un append de cada columna de genero
        df_Generos = pd.concat(list_Genero, ignore_index=True)
        
        #Limpiar columna de Genero
        df_Generos['Genero']=df_Generos['Genero'].apply(limpieza)

        #Eliminamos duplicados
        df_Generos =pd.DataFrame(df_Generos.drop_duplicates())

        #Reiniciamos el índice
        df_Generos.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')

        #Filtramos los nulos
        df_Generos=df_Generos[df_Generos.Genero.notnull()]
        #Filtrmos los vacíos 
        df_Generos=df_Generos[df_Generos['Genero']!='']

        #Agregamos una columna que va a hacer las veces de códico 
        df_Generos['Id_Genero']=np.arange(1,len(df_Generos)+1)

        #Escribiemos la dimension de Genero 
        df_Generos.to_csv("/dbfs/mnt/processed/Genero/Genero.csv",index=False)

    except Exception as e:
        print("Failed! Due to: {}".format(str(e)))
        log_error_list.append("Error: {}".format(str(e)))

#6.Escritura del dataframe de Series en processed

In [0]:
if len(log_error_list)>0:
    print("Previous Error")
else:
    try:
        Series.to_csv("/dbfs/mnt/processed/Series/Series.csv",index=False)
      
    except Exception as e:
        print("Failed! Due to: {}".format(str(e)))
        log_error_list.append("Error: {}".format(str(e)))

#6. Estado final del notebook

In [0]:
if len(log_error_list)>0:
    log_status = "Failure"
    log_error_list = list(dict.fromkeys(log_error_list))
    log_error = str(log_error_list)
    rows_read = 0
    rows_written = 0
    print(log_error)
else:
    log_status = "Success"
    log_error = ''
print(log_status)

Success